In [68]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [69]:
import jkbc.utils.files as f
import h5py as h5py
import jkbc.utils.preprocessing as prep
import numpy as np
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

In [70]:
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [100]:
data = load_obj("5mer_values_range(0, 76033)")
count = 10000
max_label_len = 70

In [101]:
def get_base():
    return random.randint(1,4)-1

def get_kmer(k):
    kmer = []
    for x in range(k):
        base = get_base()
        kmer.append(base)
    
    while True:
        base = get_base()
        kmer.append(base)
        kmer.pop(0)
        yield kmer, base

def get_reps():
    return random.randint(5, 15)

In [102]:
def get_window(kmer_dict, kmer_size, window_size=300):
    signal = []
    reference = []
    for mer, base in get_kmer(kmer_size):
        reference.append(base+1)
        reps = get_reps()
        mer_string = ''.join([str(x) for x in mer])
        value =  float(kmer_dict[mer_string])
        signal += [value] * reps

        if len(signal) > window_size:
            return signal[:window_size], reference, len(reference)

In [103]:
x, y, y_lengths = [], [], []
for _ in range(count):
    x_, y_, y_lengths_ = get_window(data, 5)
    x.append(x_)
    y.append(y_)
    y_lengths.append(y_lengths_)

y = prep.add_label_padding(y, max_label_len)

In [104]:
f.write_data_to_feather_file(f"data/feather-files/fake_data{count}", (np.array(x), np.array(y), y_lengths))